# Download HD google images

## Note: attribute values may change since Google updates their code constantly

In [645]:
#imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import urllib.request
import os
import time
from bs4 import BeautifulSoup

import base64
import io
from PIL import Image

In [646]:
#input what images to search
print('Input image to search for:')
search = input('> ')

print('Number of mouse scrolls:')
scrolls = int(input('> '))



Input image to search for:
Number of mouse scrolls:


In [647]:
#open google image search
driver = webdriver.Chrome('C:/Users/lambo/chromedriver.exe')
driver.get("https://www.google.com/imghp?hl=en")


C:\Users\lambo\AppData\Local\Temp/ipykernel_7272/2151603596.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/lambo/chromedriver.exe')


In [648]:
#wait at most 10 seconds for page to be loaded
driver.implicitly_wait(10)

#find search box
search_box = driver.find_element_by_xpath("//input[@aria-label='Search']")
search_box.clear()
search_box.send_keys(search)

time.sleep(1) #wait 1 second before clicking search button
#click search button
button = driver.find_element_by_xpath("//button[@aria-label='Google Search']")        
button.click()

C:\Users\lambo\AppData\Local\Temp/ipykernel_7272/581484003.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = driver.find_element_by_xpath("//input[@aria-label='Search']")
C:\Users\lambo\AppData\Local\Temp/ipykernel_7272/581484003.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button = driver.find_element_by_xpath("//button[@aria-label='Google Search']")


In [649]:
#wait page to be loaded
time.sleep(2)

#scroll down the webpage
for j in range(0,scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

#wait at most 10 seconds for page to be loaded
driver.implicitly_wait(10)

#find the images
images = driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy']")   #find the clickable images

img_link=[]

for i in range(0,len(images)-40):
    #click the images to display the link for opening the image in new tab (initially hidden if you do not click the image)
    driver.execute_script("arguments[0].click();", images[i]) 
    #get the link to open the images new tab (to mimic right clicking the image and selecting "Open link in new tab")
    clicked_image_link_temp = driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy']")
    clicked_image_link = clicked_image_link_temp[i].get_attribute('href')
    
    #open a new tab
    driver.execute_script('''window.open("https://www.google.com","_blank");''')
    driver.switch_to.window(driver.window_handles[1])   # switch to the new tab
    #time.sleep(1)
    #load the hd image in the newly created tab
    driver.get(clicked_image_link)
    #wait image to be fully loaded
    time.sleep(3)    ### ADJUST according to speed of internet
   
    #find src link of the hd image using BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    hd_link = soup.find_all('img',{'id':'imi'})   
    
    b = hd_link[0]['src']  #src can be sometimes in base64 encoded format
    
    if b.startswith('http'):
        img_link.append(b)
    else:
        z = b[b.find(r'/9'):]   #get only the base64 code
        img_link.append(z)
            
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    

C:\Users\lambo\AppData\Local\Temp/ipykernel_7272/467704424.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  images = driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy']")   #find the clickable images
C:\Users\lambo\AppData\Local\Temp/ipykernel_7272/467704424.py:21: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  clicked_image_link_temp = driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy']")


In [650]:
#create path where to save the files
#NOTE: change the path
path = rf'C:\Users\lambo\OneDrive\Documents\Python Scripts\google hd\{search}'
if not os.path.exists(path):
    os.makedirs(path)

    
for count, cat_link in enumerate(img_link):
    #downloading the photos may fail sometimes (error 403)
    try:
        if cat_link.startswith('http'):
            urllib.request.urlretrieve(cat_link, path + rf'\{search}_hd_{count}.jpg')
        else:
            Image.open(io.BytesIO(base64.b64decode(cat_link))).save(rf'C:\Users\lambo\OneDrive\Documents\Python Scripts\google hd\{search}\{search}_hd_{count}.jpg')
    except:
        continue